### fc_25 전처리코드
1. 선수명 표기 통일
2. 선수 포지션 표기 통일
3. 선수 소속팀 표기 통일
4. 국가명 표준화 작업
5. 중복 데이터 처리

In [65]:
import pandas as pd
import numpy as np
from unidecode import unidecode
import re

In [66]:
fc_df = pd.read_csv('../data/fc25_playerdata.csv', encoding='utf-8-sig')
fc_df

,name,nation,club,position,OVR,PAC,SHO,PAS,DRI,DEF,PHY
0,Kylian Mbappe,France,Real Madrid,ST,91,97,90,80,92,36,78
1,Rodri,Spain,Manchester City,CDM,91,66,80,86,84,87,85
2,Erling Haaland,Norway,Manchester City,ST,91,88,92,70,81,45,88
3,Jude Bellingham,England,Real Madrid,MOC,90,80,87,83,88,78,83
4,Vinicius Junior,Brazil,Real Madrid,LW,90,95,84,81,91,29,69
...,...,...,...,...,...,...,...,...,...,...,...
15900,Engson Singh,India,Mohun Bagan SG,MOC,47,31,45,47,52,34,45
15901,Ousmane Sylla,Senegal,Houston Dynamo,CM,47,67,38,46,45,40,55
15902,Wei Feng,China,Changchun Yatai,RB,47,57,47,38,50,43,49
15903,Won Hui Do,"Korea, South",Jeju United,CM,47,66,35,46,46,41,64


In [67]:
player_df = pd.read_csv('../data/tfm_finaldata.csv', encoding='utf-8-sig')
player_df

,name,field,position,age,nation,confederation,club,league,value_billion_krw
0,Lamine Yamal,FW,Right_Winger,18,Spain,UEFA,FC Barcelona,LaLiga,3250.0
1,Erling Haaland,FW,Striker,25,Norway,UEFA,Manchester City,PremierLeague,2925.0
2,Kylian Mbappe,FW,Striker,26,France,UEFA,Real Madrid,LaLiga,2925.0
3,Jude Bellingham,MF,Attacking_Midfielder,22,England,UEFA,Real Madrid,LaLiga,2925.0
4,Vinicius Junior,FW,Left_Winger,25,Brazil,CONMEBOL,Real Madrid,LaLiga,2762.5
...,...,...,...,...,...,...,...,...,...
5348,Vuniuci Tikomaimereke,DF,Left_Back,35,Fiji,OFC,Nadroga FC,Fiji_PremierLeague,0.8
5349,Frank Clarke,FW,Striker,27,New Zealand,OFC,Fencibles United AFC,NewZealand_FootballLeague,0.8
5350,Emori Ragata,GK,Goalkeeper,37,Fiji,OFC,Nasinu FC,Fiji_PremierLeague,0.8
5351,Kazik Swain,MF,Attacking_Midfielder,19,New Zealand,OFC,Bay Olympic FC,NewZealand_FootballLeague,0.8


- 한국인 선수들 이름 표기를 FC25 방식으로 통일 (ex. Min-jae Kim -> Kim Min Jae)

In [68]:
fc_kor = fc_df[fc_df['nation'] == 'Korea, South']
fc_kor.head() 
# transfermarkt 데이터와 한국인 선수 표기명이 다른걸 확인

,name,nation,club,position,OVR,PAC,SHO,PAS,DRI,DEF,PHY
38,Heung Min Son,"Korea, South",Spurs,ST,87,87,89,81,85,42,73
151,Kim Min Jae,"Korea, South",FC Bayern München,CB,83,76,33,57,63,85,83
542,Lee Kang In,"Korea, South",Paris SG,MOC,79,74,74,80,82,50,66
706,Hwang Hee Chan,"Korea, South",Wolves,ST,78,84,77,70,78,39,74
1187,Lee Jae Sung,"Korea, South",1. FSV Mainz 05,MOC,76,69,71,72,81,59,61


In [73]:
# KOR 이름 변환 함수
def convert_name(name):
    parts = name.strip().split()
    if len(parts) != 2: # 이름이 2단어가 아니면 변환하지 않음
        return name
    first_name = parts[0].replace('-', ' ').title()
    last_name = parts[1].title()
    return f"{last_name} {first_name}"

# 한국 국적 선수에게만 이름 변환 적용
player_df.loc[player_df['nation'] == 'Korea, South', 'name'] = (player_df.loc[player_df['nation'] == 'Korea, South', 'name'].apply(convert_name))

In [74]:
# 적용 확인
player_df[player_df['nation'] == 'Korea, South'].head()

,name,field,position,age,nation,confederation,club,league,value_billion_krw
163,Kim Min Jae,DF,Centre_Back,28,"Korea, South",AFC,Bayern Munich,Bundesliga,650.0
351,Lee Kang In,FW,Right_Winger,24,"Korea, South",AFC,Paris Saint-Germain,Ligue1,406.2
521,Son Heung Min,FW,Left_Winger,33,"Korea, South",AFC,Tottenham Hotspur,PremierLeague,325.0
1011,Hwang Hee Chan,FW,Striker,29,"Korea, South",AFC,Wolverhampton Wanderers,PremierLeague,195.0
1157,Hwang In Beom,MF,Central_Midfielder,28,"Korea, South",AFC,Feyenoord Rotterdam,Eredivisie,162.5


In [75]:
player_df.to_csv('../data/tfm_finaldata.csv', index=False, encoding='utf-8-sig') # 이름 변경된 걸로 덮어쓰기

-  fc25 데이터에서 이상한 점 확인 ('카보베르데제도'라는 국가만 한글표기 되어있음)

In [15]:
fc_df[fc_df['nation'] == '카보베르데제도'].head()

,name,nation,club,position,OVR,PAC,SHO,PAS,DRI,DEF,PHY
1613,Bruno Varela,카보베르데제도,Vitória SC,GK,75,75,73,69,76,39,73
1764,Logan Costa,카보베르데제도,Villarreal CF,CB,74,48,38,55,61,75,75
2260,Bebe,카보베르데제도,Racing de Ferrol,LM,73,81,76,70,71,41,83
2805,Nuno da Costa,카보베르데제도,Kasımpaşa,ST,72,82,69,63,72,27,65
4379,Leandro Andrade,카보베르데제도,Qarabağ FK,RW,70,75,68,63,73,29,60


In [16]:
# fc25 데이터에 '카보베르데제도' 국가명 transfermarkt기준에 맞게 재할당
fc_df['nation'] = fc_df['nation'].replace('카보베르데제도', 'Cabo Verde')
fc_df[fc_df['nation'] == 'Cabo Verde'].head()

,name,nation,club,position,OVR,PAC,SHO,PAS,DRI,DEF,PHY
1613,Bruno Varela,Cabo Verde,Vitória SC,GK,75,75,73,69,76,39,73
1764,Logan Costa,Cabo Verde,Villarreal CF,CB,74,48,38,55,61,75,75
2260,Bebe,Cabo Verde,Racing de Ferrol,LM,73,81,76,70,71,41,83
2805,Nuno da Costa,Cabo Verde,Kasımpaşa,ST,72,82,69,63,72,27,65
4379,Leandro Andrade,Cabo Verde,Qarabağ FK,RW,70,75,68,63,73,29,60


In [24]:
# 한글/비표준 국가명 nation 값 필터링
etc_nations = fc_df[~fc_df['nation'].str.match(r'^[a-zA-Z, ]+$')]['nation'].unique()
print(etc_nations)
print(len(etc_nations))


['Bosnia-Herzegovina' "Cote d'Ivoire" '적도기니' 'Guinea-Bissau' 'Curaçao'
 '피지' '카자흐스탄' '버뮤다' '안도라' '세인트루시아' '앤티가바부다' '몬세라트' '바베이도스' '뉴칼레도니아' '페로제도'
 '리히텐슈타인']
16


In [25]:
# 나머지 한글/비표준 국가명 → 표준 영어 국가명 매핑
nation_correction = {
    'Curaçao': 'Curacao',
    '적도기니': 'Equatorial Guinea',
    '피지': 'Fiji',
    '카자흐스탄': 'Kazakhstan',
    '버뮤다': 'Bermuda',
    '안도라': 'Andorra',
    '세인트루시아': 'Saint Lucia',
    '앤티가바부다': 'Antigua and Barbuda',
    '몬세라트': 'Montserrat',
    '바베이도스': 'Barbados',
    '뉴칼레도니아': 'New Caledonia',
    '페로제도': 'Faroe Islands',
    '리히텐슈타인': 'Liechtenstein',
}

fc_df['nation'] = fc_df['nation'].replace(nation_correction)

In [26]:
corrected_nations = list(nation_correction.values())
corrected_nations

['Curacao',
 'Equatorial Guinea',
 'Fiji',
 'Kazakhstan',
 'Bermuda',
 'Andorra',
 'Saint Lucia',
 'Antigua and Barbuda',
 'Montserrat',
 'Barbados',
 'New Caledonia',
 'Faroe Islands',
 'Liechtenstein']

In [27]:
fc_df.to_csv('../data/fc25_playerdata.csv', index=False, encoding='utf-8-sig') # 변경 적용된 걸로 덮어쓰기

In [76]:
# Identifier 생성 함수
def identifier(row):
    name_part = row['name'].replace(' ', '').lower() # 선수명의 공백을 없애고 소문자로 치환하는 형태로 구성
    nation_code = row['nation'][:3].upper()  # 국가명의 앞 3글자를 대문자로 치환하는 형태로 구성
    return f"{name_part}_{nation_code}"

# Identifier 컬럼 추가 적용
player_df['Identifier'] = player_df.apply(identifier, axis=1)
fc_df['Identifier'] = fc_df.apply(identifier, axis=1)

In [77]:
player_df.to_csv('../data/tfm_finaldata_id.csv', index=False, encoding='utf-8-sig')
fc_df.to_csv('../data/fc25_playerdata_id.csv', index=False, encoding='utf-8-sig')

In [78]:
fc_df = pd.read_csv('../data/fc25_playerdata_id.csv', encoding='utf-8-sig') # 파일 오타수정 후 재로드
fc_df.head()

,name,nation,club,position,OVR,PAC,SHO,PAS,DRI,DEF,PHY,Identifier
0,Kylian Mbappe,France,Real Madrid,ST,91,97,90,80,92,36,78,kylianmbappe_FRA
1,Rodri,Spain,Manchester City,CDM,91,66,80,86,84,87,85,rodri_SPA
2,Erling Haaland,Norway,Manchester City,ST,91,88,92,70,81,45,88,erlinghaaland_NOR
3,Jude Bellingham,England,Real Madrid,MOC,90,80,87,83,88,78,83,judebellingham_ENG
4,Vinicius Junior,Brazil,Real Madrid,LW,90,95,84,81,91,29,69,viniciusjunior_BRA


- FC25 데이터에도 필드 / 포지션 표준화 작업

In [79]:
# 포지션 표기명 통일화

position_mapping = {
    # Striker
    'Centre-Forward': 'Striker', 'Second Striker': 'Striker', 'ST': 'Striker', 'CF': 'Striker',

    # Winger
    'Right Winger': 'Right_Winger', 'RW': 'Right_Winger',
    'Left Winger': 'Left_Winger', 'LW': 'Left_Winger',
    'Attack': '(Uncategorized)Forwards',

    # Midfield
    'Attacking Midfield': 'Attacking_Midfielder', 'MOC': 'Attacking_Midfielder',
    'Central Midfield': 'Central_Midfielder', 'CM': 'Central_Midfielder',
    'Right Midfield': 'Side_Midfielder', 'Left Midfield': 'Side_Midfielder',
    'LM': 'Side_Midfielder', 'RM': 'Side_Midfielder',
    'Defensive Midfield': 'Defensive_Midfielder', 'CDM': 'Defensive_Midfielder',
    'Midfield': '(Uncategorized)Midfield',

    # Defender
    'Centre-Back': 'Centre_Back', 'Sweeper': 'Centre_Back', 'CB': 'Centre_Back',
    'Right-Back': 'Right_Back', 'RB': 'Right_Back',
    'Left-Back': 'Left_Back', 'LB': 'Left_Back',
    'Defender': '(Uncategorized)Defender',

    # Goalkeeper
    'Goalkeeper': 'Goalkeeper', 'GK': 'Goalkeeper'
}

fc_df['position'] = fc_df['position'].replace(position_mapping)
fc_df.head()

,name,nation,club,position,OVR,PAC,SHO,PAS,DRI,DEF,PHY,Identifier
0,Kylian Mbappe,France,Real Madrid,Striker,91,97,90,80,92,36,78,kylianmbappe_FRA
1,Rodri,Spain,Manchester City,Defensive_Midfielder,91,66,80,86,84,87,85,rodri_SPA
2,Erling Haaland,Norway,Manchester City,Striker,91,88,92,70,81,45,88,erlinghaaland_NOR
3,Jude Bellingham,England,Real Madrid,Attacking_Midfielder,90,80,87,83,88,78,83,judebellingham_ENG
4,Vinicius Junior,Brazil,Real Madrid,Left_Winger,90,95,84,81,91,29,69,viniciusjunior_BRA


In [80]:
# 추가로 전체 포지션 분류
FW = ['Striker', 'Right_Winger', 'Left_Winger', '(Uncategorized)Forwards']
MF = ['Attacking_Midfielder', 'Side_Midfielder', 'Central_Midfielder', 'Defensive_Midfielder', '(Uncategorized)Midfield']
DF = ['Centre_Back', 'Right_Back', 'Left_Back', '(Uncategorized)Defender']
GK = ['Goalkeeper']

conditions = [
    fc_df['position'].isin(FW),
    fc_df['position'].isin(MF),
    fc_df['position'].isin(DF),
    fc_df['position'].isin(GK)
]
choices = ['FW', 'MF', 'DF', 'GK']

fc_df['field'] = np.select(conditions, choices, default='Unknown')
fc_df.head()

,name,nation,club,position,OVR,PAC,SHO,PAS,DRI,DEF,PHY,Identifier,field
0,Kylian Mbappe,France,Real Madrid,Striker,91,97,90,80,92,36,78,kylianmbappe_FRA,FW
1,Rodri,Spain,Manchester City,Defensive_Midfielder,91,66,80,86,84,87,85,rodri_SPA,MF
2,Erling Haaland,Norway,Manchester City,Striker,91,88,92,70,81,45,88,erlinghaaland_NOR,FW
3,Jude Bellingham,England,Real Madrid,Attacking_Midfielder,90,80,87,83,88,78,83,judebellingham_ENG,MF
4,Vinicius Junior,Brazil,Real Madrid,Left_Winger,90,95,84,81,91,29,69,viniciusjunior_BRA,FW


In [81]:
new_column_order = ['Identifier','name','nation','club','field','position','OVR','PAC','SHO','PAS','DRI','DEF','PHY']
fc_df = fc_df[new_column_order]
fc_df.head()

,Identifier,name,nation,club,field,position,OVR,PAC,SHO,PAS,DRI,DEF,PHY
0,kylianmbappe_FRA,Kylian Mbappe,France,Real Madrid,FW,Striker,91,97,90,80,92,36,78
1,rodri_SPA,Rodri,Spain,Manchester City,MF,Defensive_Midfielder,91,66,80,86,84,87,85
2,erlinghaaland_NOR,Erling Haaland,Norway,Manchester City,FW,Striker,91,88,92,70,81,45,88
3,judebellingham_ENG,Jude Bellingham,England,Real Madrid,MF,Attacking_Midfielder,90,80,87,83,88,78,83
4,viniciusjunior_BRA,Vinicius Junior,Brazil,Real Madrid,FW,Left_Winger,90,95,84,81,91,29,69


In [82]:
fc_df.to_csv('../data/fc25_playerdata_id2.csv', index=False, encoding='utf-8-sig')

In [83]:
player_df.head()

,name,field,position,age,nation,confederation,club,league,value_billion_krw,Identifier
0,Lamine Yamal,FW,Right_Winger,18,Spain,UEFA,FC Barcelona,LaLiga,3250.0,lamineyamal_SPA
1,Erling Haaland,FW,Striker,25,Norway,UEFA,Manchester City,PremierLeague,2925.0,erlinghaaland_NOR
2,Kylian Mbappe,FW,Striker,26,France,UEFA,Real Madrid,LaLiga,2925.0,kylianmbappe_FRA
3,Jude Bellingham,MF,Attacking_Midfielder,22,England,UEFA,Real Madrid,LaLiga,2925.0,judebellingham_ENG
4,Vinicius Junior,FW,Left_Winger,25,Brazil,CONMEBOL,Real Madrid,LaLiga,2762.5,viniciusjunior_BRA


In [84]:
fc_df.head()

,Identifier,name,nation,club,field,position,OVR,PAC,SHO,PAS,DRI,DEF,PHY
0,kylianmbappe_FRA,Kylian Mbappe,France,Real Madrid,FW,Striker,91,97,90,80,92,36,78
1,rodri_SPA,Rodri,Spain,Manchester City,MF,Defensive_Midfielder,91,66,80,86,84,87,85
2,erlinghaaland_NOR,Erling Haaland,Norway,Manchester City,FW,Striker,91,88,92,70,81,45,88
3,judebellingham_ENG,Jude Bellingham,England,Real Madrid,MF,Attacking_Midfielder,90,80,87,83,88,78,83
4,viniciusjunior_BRA,Vinicius Junior,Brazil,Real Madrid,FW,Left_Winger,90,95,84,81,91,29,69


#### 중복 데이터 제거

In [85]:
# 공통된 Identifier만 추출 (양쪽에 모두 존재하는 값만)
common_ids = set(player_df['Identifier']) & set(fc_df['Identifier'])

# 양쪽 모두에서 해당 Identifier만 필터링
player_common = player_df[player_df['Identifier'].isin(common_ids)].copy()
fc_common = fc_df[fc_df['Identifier'].isin(common_ids)].copy()

In [92]:
player_common.to_csv('../data/player_common.csv', index=False, encoding='utf-8-sig')
player_common

,name,field,position,age,nation,confederation,club,league,value_billion_krw,Identifier
0,Lamine Yamal,FW,Right_Winger,18,Spain,UEFA,FC Barcelona,LaLiga,3250.0,lamineyamal_SPA
1,Erling Haaland,FW,Striker,25,Norway,UEFA,Manchester City,PremierLeague,2925.0,erlinghaaland_NOR
2,Kylian Mbappe,FW,Striker,26,France,UEFA,Real Madrid,LaLiga,2925.0,kylianmbappe_FRA
3,Jude Bellingham,MF,Attacking_Midfielder,22,England,UEFA,Real Madrid,LaLiga,2925.0,judebellingham_ENG
4,Vinicius Junior,FW,Left_Winger,25,Brazil,CONMEBOL,Real Madrid,LaLiga,2762.5,viniciusjunior_BRA
...,...,...,...,...,...,...,...,...,...,...
5100,Roy Krishna,FW,Striker,37,Fiji,OFC,Without Club,Without_Club,2.0,roykrishna_FIJ
5138,Tommy Smith,DF,Centre_Back,35,New Zealand,OFC,Without Club,Without_Club,1.6,tommysmith_NEW
5163,Clayton Lewis,MF,Central_Midfielder,28,New Zealand,OFC,Without Club,Without_Club,1.6,claytonlewis_NEW
5165,Gabriel Sloane-Rodrigues,FW,Right_Winger,18,New Zealand,OFC,Wellington Phoenix,A_League,1.6,gabrielsloane-rodrigues_NEW


In [93]:
fc_common.to_csv('../data/fc_common.csv', index=False, encoding='utf-8-sig')
fc_common

,Identifier,name,nation,club,field,position,OVR,PAC,SHO,PAS,DRI,DEF,PHY
0,kylianmbappe_FRA,Kylian Mbappe,France,Real Madrid,FW,Striker,91,97,90,80,92,36,78
1,rodri_SPA,Rodri,Spain,Manchester City,MF,Defensive_Midfielder,91,66,80,86,84,87,85
2,erlinghaaland_NOR,Erling Haaland,Norway,Manchester City,FW,Striker,91,88,92,70,81,45,88
3,judebellingham_ENG,Jude Bellingham,England,Real Madrid,MF,Attacking_Midfielder,90,80,87,83,88,78,83
4,viniciusjunior_BRA,Vinicius Junior,Brazil,Real Madrid,FW,Left_Winger,90,95,84,81,91,29,69
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15487,kamielvandeperre_BEL,Kamiel Van de Perre,Belgium,R. Union St.-G.,MF,Central_Midfielder,53,73,39,52,59,41,42
15499,emmanuelagyemang_GHA,Emmanuel Agyemang,Ghana,Wolfsberger AC,MF,Attacking_Midfielder,52,61,44,50,59,32,51
15621,ewanwilson_SCO,Ewan Wilson,Scotland,Motherwell,DF,Left_Back,52,66,29,36,49,49,59
15679,tyronowusu_SWI,Tyron Owusu,Switzerland,FC Luzern,MF,Central_Midfielder,51,60,33,51,51,52,55


In [ ]:
player_dup = player_common[player_common.duplicated('Identifier', keep=False)].sort_values(['Identifier', 'value_billion_krw'], ascending=[True, False])
player_dup.to_csv('../data/player_dup.csv', index=False, encoding='utf-8-sig')  # 데이터 확인용

In [ ]:
fc_dup = fc_common[fc_common.duplicated('Identifier', keep=False)].sort_values(['Identifier', 'OVR'], ascending=[True, False])
fc_dup.to_csv('../data/fc_dup.csv', index=False, encoding='utf-8-sig')  # 데이터 확인용

In [94]:
merged_df = pd.merge(fc_common, player_common, on='Identifier', suffixes=('_fc25', '_tfm'))
merged_df

,Identifier,name_fc25,nation_fc25,club_fc25,field_fc25,position_fc25,OVR,PAC,SHO,PAS,...,PHY,name_tfm,field_tfm,position_tfm,age,nation_tfm,confederation,club_tfm,league,value_billion_krw
0,kylianmbappe_FRA,Kylian Mbappe,France,Real Madrid,FW,Striker,91,97,90,80,...,78,Kylian Mbappe,FW,Striker,26,France,UEFA,Real Madrid,LaLiga,2925.0
1,rodri_SPA,Rodri,Spain,Manchester City,MF,Defensive_Midfielder,91,66,80,86,...,85,Rodri,MF,Defensive_Midfielder,29,Spain,UEFA,Manchester City,PremierLeague,1787.5
2,erlinghaaland_NOR,Erling Haaland,Norway,Manchester City,FW,Striker,91,88,92,70,...,88,Erling Haaland,FW,Striker,25,Norway,UEFA,Manchester City,PremierLeague,2925.0
3,judebellingham_ENG,Jude Bellingham,England,Real Madrid,MF,Attacking_Midfielder,90,80,87,83,...,83,Jude Bellingham,MF,Attacking_Midfielder,22,England,UEFA,Real Madrid,LaLiga,2925.0
4,viniciusjunior_BRA,Vinicius Junior,Brazil,Real Madrid,FW,Left_Winger,90,95,84,81,...,69,Vinicius Junior,FW,Left_Winger,25,Brazil,CONMEBOL,Real Madrid,LaLiga,2762.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3341,kamielvandeperre_BEL,Kamiel Van de Perre,Belgium,R. Union St.-G.,MF,Central_Midfielder,53,73,39,52,...,42,Kamiel Van De Perre,MF,Central_Midfielder,21,Belgium,UEFA,Union Saint-Gilloise,Jupiler_ProLeague,48.8
3342,emmanuelagyemang_GHA,Emmanuel Agyemang,Ghana,Wolfsberger AC,MF,Attacking_Midfielder,52,61,44,50,...,51,Emmanuel Agyemang,MF,Central_Midfielder,21,Ghana,CAF,Wolfsberger AC,Austria_Bundesliga,32.5
3343,ewanwilson_SCO,Ewan Wilson,Scotland,Motherwell,DF,Left_Back,52,66,29,36,...,59,Ewan Wilson,DF,Left_Back,20,Scotland,UEFA,Motherwell FC,Scottish_Premiership,11.4
3344,tyronowusu_SWI,Tyron Owusu,Switzerland,FC Luzern,MF,Central_Midfielder,51,60,33,51,...,55,Tyron Owusu,MF,Central_Midfielder,22,Switzerland,UEFA,FC Luzern,Swiss_SuperLeague,19.5


In [97]:
# Id가 같지만 field가 일치하지 않는 경우 추출
mismatch_field = merged_df[merged_df['field_fc25'] != merged_df['field_tfm']]
mismatch_field

# 제거 대상 Identifier만 추출
mismatch_names = mismatch_field['Identifier']

# 해당 Identifier를 기준으로 제거
player_common = player_common[~player_common['Identifier'].isin(mismatch_names)].reset_index(drop=True)
fc_common = fc_common[~fc_common['Identifier'].isin(mismatch_names)].reset_index(drop=True)

In [98]:
player_common

,name,field,position,age,nation,confederation,club,league,value_billion_krw,Identifier
0,Lamine Yamal,FW,Right_Winger,18,Spain,UEFA,FC Barcelona,LaLiga,3250.0,lamineyamal_SPA
1,Erling Haaland,FW,Striker,25,Norway,UEFA,Manchester City,PremierLeague,2925.0,erlinghaaland_NOR
2,Kylian Mbappe,FW,Striker,26,France,UEFA,Real Madrid,LaLiga,2925.0,kylianmbappe_FRA
3,Jude Bellingham,MF,Attacking_Midfielder,22,England,UEFA,Real Madrid,LaLiga,2925.0,judebellingham_ENG
4,Vinicius Junior,FW,Left_Winger,25,Brazil,CONMEBOL,Real Madrid,LaLiga,2762.5,viniciusjunior_BRA
...,...,...,...,...,...,...,...,...,...,...
2830,Roy Krishna,FW,Striker,37,Fiji,OFC,Without Club,Without_Club,2.0,roykrishna_FIJ
2831,Tommy Smith,DF,Centre_Back,35,New Zealand,OFC,Without Club,Without_Club,1.6,tommysmith_NEW
2832,Clayton Lewis,MF,Central_Midfielder,28,New Zealand,OFC,Without Club,Without_Club,1.6,claytonlewis_NEW
2833,Gabriel Sloane-Rodrigues,FW,Right_Winger,18,New Zealand,OFC,Wellington Phoenix,A_League,1.6,gabrielsloane-rodrigues_NEW


In [99]:
fc_common

,Identifier,name,nation,club,field,position,OVR,PAC,SHO,PAS,DRI,DEF,PHY
0,kylianmbappe_FRA,Kylian Mbappe,France,Real Madrid,FW,Striker,91,97,90,80,92,36,78
1,rodri_SPA,Rodri,Spain,Manchester City,MF,Defensive_Midfielder,91,66,80,86,84,87,85
2,erlinghaaland_NOR,Erling Haaland,Norway,Manchester City,FW,Striker,91,88,92,70,81,45,88
3,judebellingham_ENG,Jude Bellingham,England,Real Madrid,MF,Attacking_Midfielder,90,80,87,83,88,78,83
4,viniciusjunior_BRA,Vinicius Junior,Brazil,Real Madrid,FW,Left_Winger,90,95,84,81,91,29,69
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2831,kamielvandeperre_BEL,Kamiel Van de Perre,Belgium,R. Union St.-G.,MF,Central_Midfielder,53,73,39,52,59,41,42
2832,emmanuelagyemang_GHA,Emmanuel Agyemang,Ghana,Wolfsberger AC,MF,Attacking_Midfielder,52,61,44,50,59,32,51
2833,ewanwilson_SCO,Ewan Wilson,Scotland,Motherwell,DF,Left_Back,52,66,29,36,49,49,59
2834,tyronowusu_SWI,Tyron Owusu,Switzerland,FC Luzern,MF,Central_Midfielder,51,60,33,51,51,52,55


- ID + Field 조합으로 필터링 후 잔여 중복 데이터 확인

In [101]:
player_common[player_common.duplicated('Identifier', keep=False)].sort_values(['Identifier', 'value_billion_krw'], ascending=[True, False])

,name,field,position,age,nation,confederation,club,league,value_billion_krw,Identifier
422,Alexsandro,DF,Centre_Back,25,Brazil,CONMEBOL,LOSC Lille,Ligue1,325.0,alexsandro_BRA
2113,Alex Sandro,DF,Left_Back,34,Brazil,CONMEBOL,CR Flamengo,Brasileiro_SerieA,29.2,alexsandro_BRA
579,Fabinho,MF,Defensive_Midfielder,31,Brazil,CONMEBOL,Al-Ittihad Club,Saudi_ProLeague,243.8,fabinho_BRA
1340,Fabinho,MF,Defensive_Midfielder,23,Brazil,CONMEBOL,Red Bull Bragantino,Brasileiro_SerieA,81.2,fabinho_BRA
2271,Pedro Henrique,FW,Striker,28,Brazil,CONMEBOL,Yunnan Yukun,Chinese_SuperLeague,21.1,pedrohenrique_BRA
2631,Pedro Henrique,FW,Right_Winger,35,Brazil,CONMEBOL,Ceará Sporting Club,Brasileiro_SerieA,9.8,pedrohenrique_BRA


In [100]:
fc_common[fc_common.duplicated('Identifier', keep=False)].sort_values(['Identifier', 'OVR'], ascending=[True, False])

,Identifier,name,nation,club,field,position,OVR,PAC,SHO,PAS,DRI,DEF,PHY
314,davidlopez_SPA,David Lopez,Spain,Girona FC,DF,Centre_Back,80,45,60,71,66,82,75
2501,davidlopez_SPA,David Lopez,Spain,Burgos CF,DF,Centre_Back,65,65,31,46,51,63,78
1868,javilopez_SPA,Javi Lopez,Spain,Real Sociedad,DF,Left_Back,70,77,29,54,59,67,68
2585,javilopez_SPA,Javi Lopez,Spain,Adelaide United,DF,Right_Back,64,54,52,62,64,64,65
2490,kimtaehwan_KOR,Kim Tae Hwan,"Korea, South",Jeonbuk Hyundai,DF,Right_Back,65,80,51,58,63,57,76
2646,kimtaehwan_KOR,Kim Tae Hwan,"Korea, South",Jeju United,DF,Right_Back,63,67,42,54,58,58,73
188,reecejames_ENG,Reece James,England,Chelsea,DF,Right_Back,82,78,72,83,82,82,82
2388,reecejames_ENG,Reece James,England,Rotherham Utd,DF,Left_Back,66,70,50,57,65,63,64


In [102]:
player_common = ( # player_common 중복 제거: 가치 높은 선수 기준
    player_common
    .sort_values(['Identifier', 'value_billion_krw'], ascending=[True, False])
    .drop_duplicates(subset='Identifier', keep='first')
    .reset_index(drop=True)
)

fc_common = ( # fc_common 중복 제거: OVR 높은 선수 기준
    fc_common
    .sort_values(['Identifier', 'OVR'], ascending=[True, False])
    .drop_duplicates(subset='Identifier', keep='first')
    .reset_index(drop=True)
)

player_common.to_csv('../data/tfm_common_final.csv', index=False, encoding='utf-8-sig')
fc_common.to_csv('../data/fc_common_final.csv', index=False, encoding='utf-8-sig')

#### 정제된 데이터 병합

In [103]:
merged_df = pd.merge(fc_common, player_common, on='Identifier', suffixes=('_fc25', '_tfm'))
merged_df

,Identifier,name_fc25,nation_fc25,club_fc25,field_fc25,position_fc25,OVR,PAC,SHO,PAS,...,PHY,name_tfm,field_tfm,position_tfm,age,nation_tfm,confederation,club_tfm,league,value_billion_krw
0,aaroncresswell_ENG,Aaron Cresswell,England,West Ham,DF,Left_Back,74,59,59,74,...,62,Aaron Cresswell,DF,Left_Back,35,England,UEFA,Stoke City,Championship,8.1
1,aaronherrera_GUA,Aaron Herrera,Guatemala,D.C. United,DF,Right_Back,71,71,49,69,...,75,Aaron Herrera,DF,Right_Back,28,Guatemala,CONCACAF,D.C. United,MLS,32.5
2,aaronhickey_SCO,Aaron Hickey,Scotland,Brentford,DF,Right_Back,76,74,54,70,...,68,Aaron Hickey,DF,Right_Back,23,Scotland,UEFA,Brentford FC,PremierLeague,162.5
3,aaronlong_UNI,Aaron Long,United States,LAFC,DF,Centre_Back,70,78,39,54,...,80,Aaron Long,DF,Centre_Back,32,United States,CONCACAF,Los Angeles FC,MLS,24.4
4,aaronmolinas_ARG,Aaron Molinas,Argentina,Defensa,MF,Attacking_Midfielder,72,73,57,72,...,63,Aaron Molinas,MF,Attacking_Midfielder,24,Argentina,CONMEBOL,Defensa y Justicia,Liga_Profesional,48.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2827,zidaneiqbal_IRA,Zidane Iqbal,Iraq,FC Utrecht,MF,Central_Midfielder,63,68,58,61,...,56,Zidane Iqbal,MF,Central_Midfielder,22,Iraq,AFC,FC Utrecht,Eredivisie,65.0
2828,zidansertdemir_DEN,Zidan Sertdemir,Denmark,FC Nordsjælland,MF,Central_Midfielder,64,68,56,64,...,58,Zidan Sertdemir,MF,Central_Midfielder,20,Denmark,UEFA,FC Nordsjaelland,Danish_Superligaen,32.5
2829,zinedinemachach_FRA,Zinedine Machach,France,Melb. Victory,MF,Attacking_Midfielder,70,72,66,67,...,79,Zinedine Machach,MF,Attacking_Midfielder,29,France,UEFA,Melbourne Victory,A_League,11.4
2830,zionsuzuki_JAP,Zion Suzuki,Japan,Parma,GK,Goalkeeper,71,71,68,71,...,67,Zion Suzuki,GK,Goalkeeper,22,Japan,AFC,Parma Calcio 1913,Serie_A,325.0


In [104]:
print(merged_df.columns.tolist())

['Identifier', 'name_fc25', 'nation_fc25', 'club_fc25', 'field_fc25', 'position_fc25', 'OVR', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 'name_tfm', 'field_tfm', 'position_tfm', 'age', 'nation_tfm', 'confederation', 'club_tfm', 'league', 'value_billion_krw']


In [108]:
new_column_order = ['name_tfm','age','nation_tfm','position_tfm','club_tfm','league','confederation','value_billion_krw','field_tfm',
                    'OVR','PAC','SHO','PAS','DRI','DEF','PHY','Identifier','name_fc25','nation_fc25','club_fc25','field_fc25','position_fc25']

football_df = merged_df[new_column_order]
football_df.head()

,name_tfm,age,nation_tfm,position_tfm,club_tfm,league,confederation,value_billion_krw,field_tfm,OVR,...,PAS,DRI,DEF,PHY,Identifier,name_fc25,nation_fc25,club_fc25,field_fc25,position_fc25
0,Aaron Cresswell,35,England,Left_Back,Stoke City,Championship,UEFA,8.1,DF,74,...,74,74,75,62,aaroncresswell_ENG,Aaron Cresswell,England,West Ham,DF,Left_Back
1,Aaron Herrera,28,Guatemala,Right_Back,D.C. United,MLS,CONCACAF,32.5,DF,71,...,69,70,67,75,aaronherrera_GUA,Aaron Herrera,Guatemala,D.C. United,DF,Right_Back
2,Aaron Hickey,23,Scotland,Right_Back,Brentford FC,PremierLeague,UEFA,162.5,DF,76,...,70,74,74,68,aaronhickey_SCO,Aaron Hickey,Scotland,Brentford,DF,Right_Back
3,Aaron Long,32,United States,Centre_Back,Los Angeles FC,MLS,CONCACAF,24.4,DF,70,...,54,60,68,80,aaronlong_UNI,Aaron Long,United States,LAFC,DF,Centre_Back
4,Aaron Molinas,24,Argentina,Attacking_Midfielder,Defensa y Justicia,Liga_Profesional,CONMEBOL,48.8,MF,72,...,72,76,51,63,aaronmolinas_ARG,Aaron Molinas,Argentina,Defensa,MF,Attacking_Midfielder


In [109]:
football_df = football_df.drop(columns=football_df.columns[-5:])
football_df.head()

,name_tfm,age,nation_tfm,position_tfm,club_tfm,league,confederation,value_billion_krw,field_tfm,OVR,PAC,SHO,PAS,DRI,DEF,PHY,Identifier
0,Aaron Cresswell,35,England,Left_Back,Stoke City,Championship,UEFA,8.1,DF,74,59,59,74,74,75,62,aaroncresswell_ENG
1,Aaron Herrera,28,Guatemala,Right_Back,D.C. United,MLS,CONCACAF,32.5,DF,71,71,49,69,70,67,75,aaronherrera_GUA
2,Aaron Hickey,23,Scotland,Right_Back,Brentford FC,PremierLeague,UEFA,162.5,DF,76,74,54,70,74,74,68,aaronhickey_SCO
3,Aaron Long,32,United States,Centre_Back,Los Angeles FC,MLS,CONCACAF,24.4,DF,70,78,39,54,60,68,80,aaronlong_UNI
4,Aaron Molinas,24,Argentina,Attacking_Midfielder,Defensa y Justicia,Liga_Profesional,CONMEBOL,48.8,MF,72,73,57,72,76,51,63,aaronmolinas_ARG


In [115]:
football_df = football_df.rename(columns={
    'name_tfm': 'Name',
    'age': 'Age',
    'nation_tfm': 'Nation',
    'position_tfm': 'Position',
    'club_tfm': 'Club',
    'league': 'League',
    'confederation': 'Confederation',
    'value_billion_krw': 'Billion_KRW',
    'field_tfm': 'Field',
    'Identifier': 'Name_ID'
})

In [116]:
football_df.to_csv('../data/football_data.csv', index=False, encoding='utf-8-sig')
football_df

,Name,Age,Nation,Position,Club,Club.1,Confederation,Billion_KRW,Field,OVR,PAC,SHO,PAS,DRI,DEF,PHY,Name_ID
0,Aaron Cresswell,35,England,Left_Back,Stoke City,Championship,UEFA,8.1,DF,74,59,59,74,74,75,62,aaroncresswell_ENG
1,Aaron Herrera,28,Guatemala,Right_Back,D.C. United,MLS,CONCACAF,32.5,DF,71,71,49,69,70,67,75,aaronherrera_GUA
2,Aaron Hickey,23,Scotland,Right_Back,Brentford FC,PremierLeague,UEFA,162.5,DF,76,74,54,70,74,74,68,aaronhickey_SCO
3,Aaron Long,32,United States,Centre_Back,Los Angeles FC,MLS,CONCACAF,24.4,DF,70,78,39,54,60,68,80,aaronlong_UNI
4,Aaron Molinas,24,Argentina,Attacking_Midfielder,Defensa y Justicia,Liga_Profesional,CONMEBOL,48.8,MF,72,73,57,72,76,51,63,aaronmolinas_ARG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2827,Zidane Iqbal,22,Iraq,Central_Midfielder,FC Utrecht,Eredivisie,AFC,65.0,MF,63,68,58,61,70,52,56,zidaneiqbal_IRA
2828,Zidan Sertdemir,20,Denmark,Central_Midfielder,FC Nordsjaelland,Danish_Superligaen,UEFA,32.5,MF,64,68,56,64,67,49,58,zidansertdemir_DEN
2829,Zinedine Machach,29,France,Attacking_Midfielder,Melbourne Victory,A_League,UEFA,11.4,MF,70,72,66,67,74,42,79,zinedinemachach_FRA
2830,Zion Suzuki,22,Japan,Goalkeeper,Parma Calcio 1913,Serie_A,AFC,325.0,GK,71,71,68,71,76,56,67,zionsuzuki_JAP


In [114]:
football_df.to_csv('../data/football_data.csv', index=False, encoding='utf-8-sig')
football_df

,Name,Age,Nation,Position,Club,Club.1,Confederation,Billion_KRW,Field,OVR,PAC,SHO,PAS,DRI,DEF,PHY,Name_ID
0,Aaron Cresswell,35,England,Left_Back,Stoke City,Championship,UEFA,8.1,DF,74,59,59,74,74,75,62,aaroncresswell_ENG
1,Aaron Herrera,28,Guatemala,Right_Back,D.C. United,MLS,CONCACAF,32.5,DF,71,71,49,69,70,67,75,aaronherrera_GUA
2,Aaron Hickey,23,Scotland,Right_Back,Brentford FC,PremierLeague,UEFA,162.5,DF,76,74,54,70,74,74,68,aaronhickey_SCO
3,Aaron Long,32,United States,Centre_Back,Los Angeles FC,MLS,CONCACAF,24.4,DF,70,78,39,54,60,68,80,aaronlong_UNI
4,Aaron Molinas,24,Argentina,Attacking_Midfielder,Defensa y Justicia,Liga_Profesional,CONMEBOL,48.8,MF,72,73,57,72,76,51,63,aaronmolinas_ARG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2827,Zidane Iqbal,22,Iraq,Central_Midfielder,FC Utrecht,Eredivisie,AFC,65.0,MF,63,68,58,61,70,52,56,zidaneiqbal_IRA
2828,Zidan Sertdemir,20,Denmark,Central_Midfielder,FC Nordsjaelland,Danish_Superligaen,UEFA,32.5,MF,64,68,56,64,67,49,58,zidansertdemir_DEN
2829,Zinedine Machach,29,France,Attacking_Midfielder,Melbourne Victory,A_League,UEFA,11.4,MF,70,72,66,67,74,42,79,zinedinemachach_FRA
2830,Zion Suzuki,22,Japan,Goalkeeper,Parma Calcio 1913,Serie_A,AFC,325.0,GK,71,71,68,71,76,56,67,zionsuzuki_JAP


> 전처리 전 데이터
- transfermarkt 데이터: 5353개
- FC25 데이터: 15905개